# Assignment 2

In this assignment, you will write a program that plot ASCII text approximations of the [Mandelbrot set](https://en.wikipedia.org/wiki/Mandelbrot_set).


## Problems

### Problem 1

Consider the function $P_{(x,y)}$ defined as follows:

$$P_{(x,y)} (u, v) = (u^2 − v^2 + x, 2uv + y)$$

We define the orbit $O(x, y)$ of a point $(x, y)$ to be an infinite list of items:

$$O(x, y) = \{(0, 0), P_{(x,y)}(0, 0), P_{(x,y)}(P_{(x,y)}(0, 0)), P_{(x,y)}(P_{(x,y)}(P_{(x,y)}(0, 0))), \ldots \}$$

In other words, the nth entry of the list $O(x, y)$ is the $P_{(x,y)}$ function composed with itself n times and then applied to $(0, 0)$.

Define a F# function `orbit` that takes a single point $(x,y)$ and index $i$ as an arguments and returns an $i^{th}$ element of the infinite list corresponding to $O(x,y)$.

- You may want to define a helper function corresponding to $P_{(x,y)}$.



### Problem 2

Define a recursive function `disp` that takes two arguments:

- a number `d`, and
- a list of tuples `pairs`.

Every tuple in this input list consists of a number followed by a character, and
you can assume the input list is always in ascending order. For example, a possible
input list might be:

```fsharp
[(0.15, '#'), (0.5, 'x'), (1.0, '.')]
```

The function `disp` should return the character from the list that corresponds
to the **smallest number** on the list that is greater than the input `d`, and
if `d` is  larger than all the number in the list, `disp` should return a space
character, ' '. For example,

```fsharp
disp 0.01 [(0.15, '#'), (0.5, 'x'), (1.0, '.')] = '#'
disp 0.4 [(0.15, '#'), (0.5, 'x'), (1.0, '.')]  = 'x'
disp 100 [(0.15, '#'), (0.5, 'x'), (1.0, '.')]  = ' '
```


### Problem 3

One way to approximate the Mandelbrot set is to consider a certain element
within the orbit of every point on the plane (such as the 12th element) and
to check whether that element is within a a certain distance from the origin;
if it is within this distance, then a non-blank character should be printed,
else it should be left blank. You should use the following function to calculate
distances of points from the origin:
```fsharp
let norm (x,y) = x*x + y*y
```
These distance values can then be used with `disp` and `orbit` to turn points on
the plane into appropriate ASCII characters within an ASCII plot of
the Mandelbrot set.

Define a function `mandelbrot` that takes three arguments:

- the resolution of the approximation, `r` (used with the `plane` function),
- the index of the elements, `i`, to check in the orbit lists of the points, and
- the formatting list, `l` (to be used with the `disp` function).

This function should return a list of characters that corresponds to a picture approximating the shape of the Mandelbrot set on the plane.

You will need to combine the `split`, `plane`, `disp`, and `orbit` functions
appropriately; list comprehensions are allowed.

Once you’ve defined the function `mandelbrot`, you can generate an ASCII version of
an approximation of the Mandelbrot set by evaluating the expression:

```fsharp
mandelbrot 17 12 disp_symbols |> printfn "%s";;
```

## Submission

Fill the missing parts of the functions (see below) and submit through the Blackboard.

## Lab Source Core

In [6]:
let disp_symbols = [(0.15, '#'); (0.5, 'x'); (1.0, '.')]

let str chs = List.fold (fun str x -> str + x.ToString()) "" chs

let plane r = [
    for y in -1.0 .. 1.0/r .. 1.0 do
        for x in -2.0 .. 1.0/r .. 1.0 do
            yield (x, y) ]
            
let norm (x,y) :float = x*x + y*y

# Solution

In [10]:
let split n ch xs =
    let rec splitloop c ch xs =
        match xs with
        | [] -> []
        | x::xs when c = 1 -> x :: ch :: splitloop n ch xs
        | x::xs -> x :: splitloop (c-1) ch xs
    splitloop n ch xs

let P (x,y) = fun (u,v) -> (u*u - v*v + x, 2.0*u*v+y)

let rec orbit (x,y) i =    
    match i with
    | 0 -> (0.0, 0.0)
    | _ -> orbit (x,y) (i-1) |> P (x,y)

let disp xs (d:float) =
    let rec mintup xs =
        match xs with
        | (v,c)::xs when v < d -> mintup xs
        | x::xs -> x |> snd
        | [] -> ' '
    mintup xs

let mandelbrot r i = 
    [ for p in plane (float r) do
          yield orbit p i |> norm |> disp disp_symbols
    ] |> split (3*r+1) '\n' |> str

In [11]:
mandelbrot 17 12 |> printfn "%s"

##############                              ########
############                    .            #######
###########                     #.            ######
#########                     ####x           ######
########                      ####x            #####
######                   .    x###              ####
#####                    x  .xx###...           ####
####                     xx..xxx#xxxxx ##       ####
###                      xxxxxxxxxxxxx##.        ###
##                    ..x#xxxxxx#xxxxx#x         ###
#                      xx##xxx#######xxxx        ###
                      xxxxx###########xx..#      ###
              x. #.   .xxxx###########xxx         ##
              ######  .xxxx############x#.        ##
             #######xxxxxx#############xx         ##
             x#######xxxxx#############xx         ##
           ############xx##############x          ##
 # ##x  ###############xx##############             
           ############xx##############x      